In [1]:
import pandas as pd
from decimal import Decimal

### USDC ETH (destakers.pickle) remove_duplicates > to_df
- logs of all who destaked from USDc ETH Pool
- `destakers`

In [2]:
df = pd.DataFrame(pd.read_pickle('data/destakers.pickle'))
print('shape: ',df.shape)
df.drop_duplicates(subset=['transactionHash'], inplace=True)
df.reset_index(drop=True, inplace=True)
print('shape: ',df.shape)

df['increased'] = 0 ## decreased = 0
df['tokenID'] = df['topics'].apply(lambda x: int(x[1],16))
df['amtETH'] = df['data'].apply(lambda x: int(x[66+64:67+64+64],16))
df = df[['transactionHash','increased', 'tokenID', 'amtETH']]
destakers = df.copy()

shape:  (61913, 11)
shape:  (53382, 11)


### USDC transfers from Uniswap V3 to Users who decresed liquidity
- to get user wallets who dec liquidity
- `destakers_usdc`

In [3]:
df = pd.DataFrame(pd.read_pickle('data/destakers_usdc.pickle'))
print('shape: ',df.shape)
df.drop_duplicates(subset=['transactionHash'], inplace=True)
df.reset_index(drop=True, inplace=True)
print('shape: ',df.shape)

# df['increased'] = 0 ## decreased = 0
# df['tokenID'] = df['topics'].apply(lambda x: int(x[1],16))
# df['amtETH'] = df['data'].apply(lambda x: int(x[66+64:67+64+64],16))
# df = df[['transactionHash','increased', 'tokenID', 'amtETH']]
destakers_usdc = df.copy()
destakers_usdc;

shape:  (61083, 11)
shape:  (61077, 11)


### USDC ETH (stakers.pickle) remove duplicates > to_df
- logs of all who staked to USDC ETH Pool
- `stakers`

In [4]:
df = pd.DataFrame(pd.read_pickle('data/stakers.pickle'))
print('shape: ',df.shape)
df.drop_duplicates(subset=['transactionHash'], inplace=True)
df.reset_index(drop=True, inplace=True)
print('shape: ',df.shape)

df['increased'] = 1
df['amtETH'] = df['data'].apply(lambda x: int(x[66+64+64:67+64+64+64],16))
df = df[['transactionHash','increased', 'amtETH']]
stakers = df.copy()
stakers;

shape:  (64761, 11)
shape:  (61500, 11)


### ERC721 Uniswap Positions NFT Transfers
- mostly data are transfers when minting, can be used to match txHash and get liquidity_provider address
- `uniswap_positions`

In [5]:
df = pd.DataFrame(pd.read_pickle('data/uniswap_positions.pickle'))
df.drop(columns=['gas', 'gasUsed', 'gasPrice', 'tokenName', 'tokenSymbol','blockHash'], inplace=True)
print('shape: ',df.shape)
df.drop_duplicates(subset=['hash'], inplace=True)
df[['blockNumber', 'tokenID']] = df[['blockNumber', 'tokenID']].apply(pd.to_numeric)
df.sort_values(by=['tokenID'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.rename(columns={'hash':'transactionHash'}, inplace=True)
print('shape: ',df.shape)

shape:  (320000, 13)
shape:  (317862, 13)


In [6]:
uniswap_positions = df.copy()

### Merge stakers with erc721 transfers
- `ledger_staked_sum`

In [7]:
df = pd.merge(stakers, uniswap_positions, on='transactionHash')
df.shape

(42142, 15)

In [8]:
ledger = pd.DataFrame(df.groupby('to', sort=False)['amtETH'].sum())
ledger = ledger.sort_values(['amtETH'], ascending=False)
ledger['amtETH'] = ledger['amtETH'].apply(lambda x: Decimal(x) / Decimal(10 ** 18))
ledger_staked_sum = ledger.copy()
ledger_staked_sum.shape

(22782, 1)

In [9]:
ledger_staked_sum

,amtETH
to,
0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,9897204.544966678708686163
0x3bb152e1758659663d6ae8377d054be015e191f3,137378.999999528094103971
0x741aa7cfb2c7bf2a1e7d4da2e3df6a56ca4131f3,96949.557316869609300479
0x60d552f33b029547e39ff1e82ad26649ccd3b88a,74716.999997607720917658
0x72a916702bd97923e55d78ea5a3f413dec7f7f85,51170.768772326949945256
...,...
0xb6f3f2ba626f418de4902d804959555427366a66,0
0x317a98cd9ff50f5b396893e4fc2636a34631acac,0
0xc4ed448e7d7bdd954e943954459017be63584f69,0


### Merge `destakers` and `destakers_usdc`
- `ledger_destaked_sum`

In [10]:
df = pd.merge(destakers, destakers_usdc, on='transactionHash')
df['wallet'] = df['topics'].apply(lambda x: '0x'+ x[2][-40:])
df.shape

(37233, 15)

In [11]:
ledger = pd.DataFrame(df.groupby('wallet', sort=False)['amtETH'].sum())
ledger = ledger.sort_values(['amtETH'], ascending=False)
ledger['amtETH'] = ledger['amtETH'].apply(lambda x: Decimal(-x) / Decimal(10 ** 18))
ledger_destaked_sum = ledger.copy()
ledger_destaked_sum.shape

(10416, 1)

In [12]:
ledger_destaked_sum

,amtETH
wallet,
0x3bb152e1758659663d6ae8377d054be015e191f3,-91300.912410926393479572
0x60d552f33b029547e39ff1e82ad26649ccd3b88a,-77232.733569159578689477
0x741aa7cfb2c7bf2a1e7d4da2e3df6a56ca4131f3,-73893.765481660425117379
0xec890fca5c79a443f53849bc658947a99b653a4c,-41549.607126635859691446
0x72a916702bd97923e55d78ea5a3f413dec7f7f85,-35105.637262920545107928
...,...
0xd361cc196f33534ed8176ab1f189983265212fd0,0
0x1bad528c8feea7c9bd0aa40944d655c50ac6ae2d,0
0x80b678b5d1af00935716f9652b7e1448889876ba,0


### Merge both ledger

In [41]:
d_staked = dict(zip(ledger_staked_sum.index, ledger_staked_sum.amtETH))

In [42]:
len(d_staked)

22782

In [43]:
d_destaked = dict(zip(ledger_destaked_sum.index, ledger_destaked_sum.amtETH))

In [44]:
len(d_destaked)

10416

In [53]:
for key, value in d_destaked.items():
    if key in d_staked.keys():
        d_staked[key] = d_staked[key] + value
    else:
        d_staked[key] = value
    

In [54]:
len(d_staked)

23121

In [61]:
df = pd.DataFrame.from_dict(d_staked, orient = 'index')

In [63]:
df.columns = ['amtETH']

In [64]:
df.sort_values(['amtETH'], ascending=False)

,amtETH
0xa57bd00134b2850b2a1c55860c9e9ea100fdd6cf,9897204.544966678708686163
0x9799b475dec92bd99bbdd943013325c36157f383,30063.935564724452090698
0xb136f3756028dc75de4dd24bdc0c2f0270e18b3c,18003.999999999999950388
0xdc848a72842d943b87926ae27ee05f1e949ec931,8996.80150283464141695
0x763bf487d386afbf9c476e047d37b74636b9e831,7085.429999999999999191
...,...
0x46a0b4fa58141aba23185e79f7047a7dfd0ff100,-34805.159289152782821128
0x3bb152e1758659663d6ae8377d054be015e191f3,-45222.824822324692855173
0x741aa7cfb2c7bf2a1e7d4da2e3df6a56ca4131f3,-50837.973646451240934279
0x60d552f33b029547e39ff1e82ad26649ccd3b88a,-79748.467140711436461296
